In [1]:
from weibo_scrapper import weibo_scrapper
import time

In [2]:
# specifying the variables used for testing
begin_date ='2018-12-28'
end_date = '2019-1-4'
username = 'ruoyzhang@outlook.com'
password = 'Cyfloel1.1.'
search_keyword = '人工智能'
save_dir = '../master_thesis_data/weibo'

In [3]:
scrape_this = weibo_scrapper(username=username, password=password,
                            save_dir=save_dir)

In [6]:
scrape_this.login_weibo()
time.sleep(20)

loading 'https://www.weibo.com/login.php'
inputting username and password
login successful
End of login


In [7]:
begin = time.time()
scrape_this.scrape(begin_date=begin_date, end_date=end_date, search_keyword=search_keyword)
end = time.time()
time_lapsed = (end - begin)/60
print('time lapsed for 50 pages: ', str(time_lapsed))

navigating to Weibo adv search page
waiting for page to load ...
specifying search criterion
waiting for results to load ...
there are  50  of results
now scrapping ...
page  1  complete
page  2  complete
page  3  complete
page  4  complete
page  5  complete
page  6  complete
page  7  complete
page  8  complete
page  9  complete
page  10  complete
page  11  complete
page  12  complete
page  13  complete
page  14  complete
page  15  complete
page  16  complete
page  17  complete
page  18  complete
page  19  complete
page  20  complete
page  21  complete
page  22  complete
page  23  complete
page  24  complete
page  25  complete
page  26  complete
page  27  complete
page  28  complete
page  29  complete
page  30  complete
page  31  complete
page  32  complete
page  33  complete
page  34  complete
page  35  complete
page  36  complete
page  37  complete
page  38  complete
page  39  complete
page  40  complete
page  41  complete
page  42  complete
page  43  complete
page  44  complete
page

Ok on a GUI enabled machine with my shitty internet connection, in order to scrape 1 week of data we'll need 9 mins of exec time

In [10]:
print('In order to gather 5 years of data, we\'ll need',str(9 * 52 * 5 / 60), 'hours')

In order to gather 5 years of data, we'll need 39.0 hours


In [11]:
print('In order to gather 3 years of data, we\'ll need',str(9 * 52 * 3 / 60), 'hours')

In order to gather 3 years of data, we'll need 23.4 hours


So 1.5 days if we want 5 years of data and a day if we want 3 days of data

Let's see if the data we scrapped are in the correct shape

In [3]:
import pickle
import os
from tqdm import tqdm

In [4]:
file_dirs = os.listdir(save_dir)

In [5]:
file_dirs

['4_0_2019_text.pickle', '4_0_2019_date.pickle']

In [6]:
with open(os.path.join(save_dir, file_dirs[0]), 'rb') as handle:
    tweets = pickle.load(handle)
with open(os.path.join(save_dir, file_dirs[1]), 'rb') as handle:
    dates = pickle.load(handle)

In [7]:
len(tweets), len(dates)

(934, 934)

YUUUUUUS! OK not exactly 1K but close enough

In [8]:
set(dates)

{datetime.datetime(2019, 1, 4, 0, 0)}

Although only tweets of one particular dates were recorded, since there are simply too many tweets, also we need to estimate the proportion of duplicated tweets, let's use this dataset for estimation

In [41]:
# rough estimation

clusters = []

for i, tweet_0 in tqdm(enumerate(tweets)):
    if sum([i in clus for clus in clusters]) == 0:
        clusters.append([i])
    else:
        continue
    for j,tweet_1 in enumerate(tweets[i+1:]):
        if tweet_0 == tweet_1:
            clusters[-1].append(i+j+1)

934it [00:00, 14435.19it/s]


OK since the big majority of the duplicated tweets are exactly the same (C&P job), this will help us out A LOT in the deduplication process, however, as we can see quite clearly, we're running on quite a bit of duplicated tweets

In [44]:
len(clusters)

580

We only have 580 original tweets

In [46]:
dup_clusters = [clus for clus in clusters if len(clus) > 1]

In [47]:
from collections import Counter
Counter([len(clus) for clus in dup_clusters])

Counter({2: 33,
         3: 6,
         4: 2,
         5: 1,
         6: 2,
         7: 1,
         9: 1,
         15: 1,
         35: 1,
         61: 1,
         168: 1})

In [48]:
sum([len(clus) for clus in dup_clusters])

404

In [50]:
print('the number of duplicated tweets are', len(dup_clusters), 'they represent a volume of', len(clusters), 'tweets')

the number of duplicated tweets are 50 they represent a volume of 580 tweets


We should consider removing these duplicated tweets since the big majority of them are bot tweets and can lead to severe bias in our data, esepcially considering their over representation

We'll worry about the deduplication later, let's move on